In [142]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
from fastai.tabular.all import *
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from dtreeviz.trees import *
from glob import glob


pd.options.display.max_rows = 60


In [143]:
save_dir = Path("../market/save")


# Load all files matching the pattern into a list of DataFrames
all_files = glob(str(save_dir / "rdf_output_*_CSCO.csv"))
print(f"Found {len(all_files)} files matching the pattern: " + str(all_files))
df_list = [pd.read_csv(file) for file in all_files]

# Concatenate all DataFrames
odf = pd.concat(df_list, ignore_index=True)
odf = odf.sort_values(by='ts_event').reset_index(drop=True)

#start_date = "2025-01-07 00:30:00"
#end_date = "2025-01-08 00:59:56"
#filtered_odf = odf.iloc[8000:20000]   # 1000 points from previous day and the current day
filtered_odf = odf
#odf[(odf['ts_event'] >= start_date) & (odf['ts_event'] <= end_date)]
filtered_odf

Found 14 files matching the pattern: ['../market/save/rdf_output_20250326_CSCO.csv', '../market/save/rdf_output_20250422_CSCO.csv', '../market/save/rdf_output_20250416_CSCO.csv', '../market/save/rdf_output_20250424_CSCO.csv', '../market/save/rdf_output_20250331_CSCO.csv', '../market/save/rdf_output_20250328_CSCO.csv', '../market/save/rdf_output_20250415_CSCO.csv', '../market/save/rdf_output_20250421_CSCO.csv', '../market/save/rdf_output_20250417_CSCO.csv', '../market/save/rdf_output_20250402_CSCO.csv', '../market/save/rdf_output_20250425_CSCO.csv', '../market/save/rdf_output_20250401_CSCO.csv', '../market/save/rdf_output_20250327_CSCO.csv', '../market/save/rdf_output_20250423_CSCO.csv']


,ts_event,rtype,publisher_id,instrument_id,action,side,depth,price,size,flags,...,rolling_5min_ask_cnt,mid,bid_weight,ask_weight,bid_weight_log,ask_weight_log,is_buy,ideal_price,ideal_price_spread,hedge_stop_prc
0,2025-03-26 09:30:00.246697671-04:00,10,2,3745,A,B,1,61.00,200,130,...,0.0,61.050,2.964161e+06,3.033622e+06,14.902104,14.925268,True,61.36,-4.666667,61.02
1,2025-03-26 09:30:00.246697671-04:00,10,2,3745,A,B,1,61.00,200,130,...,0.0,61.050,2.964161e+06,3.033622e+06,14.902104,14.925268,True,61.02,0.000000,61.08
2,2025-03-26 09:30:00.247829449-04:00,10,2,3745,A,B,4,60.99,1,130,...,0.0,61.070,3.468709e+06,3.696815e+06,15.059293,15.122982,True,61.36,-4.333333,61.04
3,2025-03-26 09:30:00.247829449-04:00,10,2,3745,A,B,4,60.99,1,130,...,0.0,61.070,3.468709e+06,3.696815e+06,15.059293,15.122982,True,61.02,-0.333333,61.10
4,2025-03-26 09:30:00.258842710-04:00,10,2,3745,A,A,1,61.11,1,128,...,0.0,61.085,4.130410e+06,4.223947e+06,15.233887,15.256281,True,61.32,-7.333333,61.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139995,2025-04-25 15:59:58.801847100-04:00,10,2,3745,A,B,1,56.68,100,130,...,1332.0,56.695,1.506952e+08,4.572328e+08,18.830770,19.940703,True,56.69,1.000000,56.69
139996,2025-04-25 15:59:58.807725709-04:00,10,2,3745,A,A,5,56.74,2100,130,...,1332.0,56.685,2.532828e+08,3.276137e+08,19.350017,19.607346,True,56.69,-0.000000,56.68
139997,2025-04-25 15:59:58.807725709-04:00,10,2,3745,A,A,5,56.74,2100,130,...,1332.0,56.685,2.532828e+08,3.276137e+08,19.350017,19.607346,True,56.69,1.000000,56.69
139998,2025-04-25 15:59:59.488572061-04:00,10,2,3745,A,A,1,56.70,100,128,...,1342.0,56.685,2.313273e+08,3.747093e+08,19.259344,19.741661,True,56.69,1.000000,56.69


In [144]:
odf.shape

(140000, 59)

#### Simplify the data further

In [145]:
columns_to_keep = ['instrument_id', #'symbol',
                   'ns_since_open', 
      'day_of_week',
      'day_in_year',
      'is_third_friday',
      'is_first_monday_after_third_friday',
      'is_day_before_third_friday',

       #'spread', 
       #'mid', 
       #'bid_px_00', 'ask_px_00',
       'bid_weight', 'ask_weight', 'bid_weight_log',
       'ask_weight_log', 
       #'traded_bid_size', 'traded_ask_size', 'is_trade_bid', 'is_trade_ask', 
       'rolling_30s_bid_size', 'rolling_30s_ask_size',
       'rolling_30s_bid_cnt', 'rolling_30s_ask_cnt', 'rolling_5min_bid_size',
       'rolling_5min_ask_size', 'rolling_5min_bid_cnt', 'rolling_5min_ask_cnt',
       #'hedge_buy_stop_idx', 'hedge_sell_stop_idx', 
       'is_buy', 
       #'hedge_stop_prc'
]
dep_vars = ['ideal_price_spread'] 

#       'ideal_buy_price', 'ideal_sell_price', 
#       'ideal_buy_price_spread', 'ideal_sell_price_spread']

extra_columns = [col for col in columns_to_keep if col not in odf.columns]

ignored_columns = [col for col in odf.columns if col not in columns_to_keep + dep_vars]
ignored_columns, extra_columns



(['ts_event',
  'rtype',
  'publisher_id',
  'action',
  'side',
  'depth',
  'price',
  'size',
  'flags',
  'ts_in_delta',
  'sequence',
  'bid_px_00',
  'ask_px_00',
  'bid_ct_00',
  'ask_ct_00',
  'bid_ct_01',
  'ask_ct_01',
  'bid_ct_02',
  'ask_ct_02',
  'bid_ct_03',
  'ask_ct_03',
  'bid_ct_04',
  'ask_ct_04',
  'bid_ct_05',
  'ask_ct_05',
  'bid_ct_06',
  'ask_ct_06',
  'bid_ct_07',
  'ask_ct_07',
  'bid_ct_08',
  'ask_ct_08',
  'bid_ct_09',
  'ask_ct_09',
  'symbol',
  'spread',
  'mid',
  'ideal_price',
  'hedge_stop_prc'],
 [])

In [146]:
rdf = filtered_odf[columns_to_keep + dep_vars]
rdf.reset_index(drop=True, inplace=True)
rdf 


,instrument_id,ns_since_open,day_of_week,day_in_year,is_third_friday,is_first_monday_after_third_friday,is_day_before_third_friday,bid_weight,ask_weight,bid_weight_log,...,rolling_30s_bid_size,rolling_30s_ask_size,rolling_30s_bid_cnt,rolling_30s_ask_cnt,rolling_5min_bid_size,rolling_5min_ask_size,rolling_5min_bid_cnt,rolling_5min_ask_cnt,is_buy,ideal_price_spread
0,3745,2.466983e+08,2,85,False,False,False,2.964161e+06,3.033622e+06,14.902104,...,297.0,0.0,4.0,0.0,297.0,0.0,4.0,0.0,True,-4.666667
1,3745,2.466983e+08,2,85,False,False,False,2.964161e+06,3.033622e+06,14.902104,...,297.0,0.0,4.0,0.0,297.0,0.0,4.0,0.0,True,0.000000
2,3745,2.478299e+08,2,85,False,False,False,3.468709e+06,3.696815e+06,15.059293,...,317.0,0.0,6.0,0.0,317.0,0.0,6.0,0.0,True,-4.333333
3,3745,2.478299e+08,2,85,False,False,False,3.468709e+06,3.696815e+06,15.059293,...,317.0,0.0,6.0,0.0,317.0,0.0,6.0,0.0,True,-0.333333
4,3745,2.588434e+08,2,85,False,False,False,4.130410e+06,4.223947e+06,15.233887,...,425.0,0.0,11.0,0.0,425.0,0.0,11.0,0.0,True,-7.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139995,3745,2.339880e+13,4,115,False,False,False,1.506952e+08,4.572328e+08,18.830770,...,66888.0,64901.0,383.0,301.0,183915.0,239885.0,1181.0,1332.0,True,1.000000
139996,3745,2.339881e+13,4,115,False,False,False,2.532828e+08,3.276137e+08,19.350017,...,66888.0,64901.0,383.0,301.0,183915.0,239885.0,1181.0,1332.0,True,-0.000000
139997,3745,2.339881e+13,4,115,False,False,False,2.532828e+08,3.276137e+08,19.350017,...,66888.0,64901.0,383.0,301.0,183915.0,239885.0,1181.0,1332.0,True,1.000000
139998,3745,2.339949e+13,4,115,False,False,False,2.313273e+08,3.747093e+08,19.259344,...,80989.0,67040.0,396.0,311.0,198016.0,242024.0,1194.0,1342.0,True,1.000000


In [147]:
cont_cols,cat_cols = cont_cat_split(rdf, max_card=9000, dep_var=dep_vars)
print("********* Continuous columns")
[print(x) for x in cont_cols] 
print("********* Categorical columns")
_= [print(x) for x in cat_cols] 
#cont_cols

********* Continuous columns
ns_since_open
bid_weight
ask_weight
bid_weight_log
ask_weight_log
rolling_30s_bid_size
rolling_30s_ask_size
rolling_30s_bid_cnt
rolling_30s_ask_cnt
rolling_5min_bid_size
rolling_5min_ask_size
rolling_5min_bid_cnt
rolling_5min_ask_cnt
********* Categorical columns
instrument_id
day_of_week
day_in_year
is_third_friday
is_first_monday_after_third_friday
is_day_before_third_friday
is_buy


In [148]:
rdf.columns

Index(['instrument_id', 'ns_since_open', 'day_of_week', 'day_in_year',
       'is_third_friday', 'is_first_monday_after_third_friday',
       'is_day_before_third_friday', 'bid_weight', 'ask_weight',
       'bid_weight_log', 'ask_weight_log', 'rolling_30s_bid_size',
       'rolling_30s_ask_size', 'rolling_30s_bid_cnt', 'rolling_30s_ask_cnt',
       'rolling_5min_bid_size', 'rolling_5min_ask_size',
       'rolling_5min_bid_cnt', 'rolling_5min_ask_cnt', 'is_buy',
       'ideal_price_spread'],
      dtype='object')

In [149]:
rdf.shape

(140000, 21)

## Scikit Learn

In [170]:
from sklearn.ensemble import HistGradientBoostingRegressor, GradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

ts_cv = TimeSeriesSplit(n_splits=3, max_train_size=500000, test_size=10000)
splits = list(ts_cv.split(rdf[columns_to_keep], rdf[dep_vars]))
splits
for train,test in splits:
    print(f"Train: {train.shape}, Test: {test.shape}")

Train: (110000,), Test: (10000,)
Train: (120000,), Test: (10000,)
Train: (130000,), Test: (10000,)


In [171]:
def profit_one_row(row):
    spread_currency = (row['ask_px_00']-row['bid_px_00']) / 2
    if row['Prediction'] > row['ideal_price_spread']:
        return -row['Prediction'] * spread_currency
    else:
        return -abs(row['hedge_stop_prc'] - (row['bid_px_00'] if row['is_buy'] else row['ask_px_00']))

def build_profit_df(predictions, x_test, y_test):
    #all = pd.concat([x_test, y_test], axis=1)
    all = odf.loc[x_test.index]
    all['Prediction'] = predictions

    # here assuming that, once the stop loss is hit, the trade will close at the opposite price.
    # stop loss is 2 ticks on the bid side so the profit is -0.02 - spread
    # prediction is in multiple of the spread 
    all['Profit'] = all.apply(profit_one_row , axis=1)
    return all 



In [172]:

for train,test in splits:
    est = HistGradientBoostingRegressor(learning_rate=0.1, max_depth=6)
    traindf = rdf.loc[train]
    x = traindf[columns_to_keep]
    y = traindf[dep_vars[0]]
    est.fit(x, y)

    testdf = rdf.loc[test]
    tx = testdf[columns_to_keep]
    ty = testdf[dep_vars[0]]
    accuracy = est.score(tx, ty)
    predictions = est.predict(tx)
    
    mse = mean_squared_error(ty, predictions)
    print(f"Accuracy: {accuracy}")
    print(f"Mean Squared Error: {mse}")

    profit_df = build_profit_df(predictions, tx, ty)
    print(f"Profit: {profit_df['Profit'].sum()}")
    print(f"Avg Profit: {profit_df['Profit'].sum() / len(profit_df)} positive: {len(profit_df[profit_df['Profit'] > 0])*100/len(profit_df):.1f}%")
    print()




Accuracy: 0.06130976354200135
Mean Squared Error: 616.4526243992107
Profit: 621.6309045808292
Avg Profit: 0.06216309045808292 positive: 50.8%

Accuracy: -0.641057408917479
Mean Squared Error: 305.0969319168356
Profit: 245.25801547072717
Avg Profit: 0.024525801547072717 positive: 22.5%

Accuracy: 0.17390236249474
Mean Squared Error: 145.69892635569335
Profit: 351.95489593371815
Avg Profit: 0.03519548959337181 positive: 45.3%



In [173]:
profit_df[['Profit','Prediction', 'ideal_price_spread', 'spread', 'bid_px_00', 'ask_px_00', 'is_buy', 'hedge_stop_prc']].head(20)

,Profit,Prediction,ideal_price_spread,spread,bid_px_00,ask_px_00,is_buy,hedge_stop_prc
130000,0.024184,-4.836894,-22.000000,0.000178,56.24,56.25,True,56.24
130001,0.024184,-4.836894,-5.000000,0.000178,56.24,56.25,True,56.25
130002,-0.030000,-4.909244,-1.666667,0.000533,56.24,56.27,True,56.27
130003,0.073639,-4.909244,-6.666667,0.000533,56.24,56.27,True,56.24
130004,-0.020000,-5.410655,-1.000000,0.000356,56.21,56.23,True,56.23
130005,0.054107,-5.410655,-12.000000,0.000356,56.21,56.23,True,56.21
130006,-0.010000,-7.316792,-2.000000,0.000178,56.21,56.22,True,56.22
130007,0.036584,-7.316792,-25.000000,0.000178,56.21,56.22,True,56.21
130008,0.038131,-7.626217,-25.000000,0.000178,56.21,56.22,True,56.21
130009,-0.010000,-7.626217,-2.000000,0.000178,56.21,56.22,True,56.22


In [174]:
# Get feature importances
importances = est.feature_importances_

# Get feature names if available
feature_names = getattr(x, "columns", [f"Feature {i}" for i in range(x.shape[1])])

# Sort features by importance
indices = np.argsort(importances)

# Plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(indices)), importances[indices], align='center')
plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
plt.xlabel("Relative Importance")
plt.title("Feature Importance in HistGradientBoostingRegressor")
plt.tight_layout()
plt.show()

AttributeError: 'HistGradientBoostingRegressor' object has no attribute 'feature_importances_'